<a href="https://colab.research.google.com/github/pagalavan-sivakumar/BitcoinPricePredictor/blob/main/BitcoinPricePredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Import Libraires

In [24]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler

2. Importing Data

In [6]:
from google.colab import files
uploaded = files.upload()

Saving BTC-USD.csv to BTC-USD (1).csv


In [7]:
data = pd.read_csv (r'BTC-USD.csv')
print (data)

            Date          Open  ...     Adj Close        Volume
0     2014-09-17    465.864014  ...    457.334015  2.105680e+07
1     2014-09-18    456.859985  ...    424.440002  3.448320e+07
2     2014-09-19    424.102997  ...    394.795990  3.791970e+07
3     2014-09-20    394.673004  ...    408.903992  3.686360e+07
4     2014-09-21    408.084991  ...    398.821014  2.658010e+07
...          ...           ...  ...           ...           ...
2632  2021-12-01  56907.964844  ...  57229.828125  3.685820e+10
2633  2021-12-02  57217.371094  ...  56477.816406  3.237997e+10
2634  2021-12-03  56509.164063  ...  53598.246094  3.978913e+10
2635  2021-12-04  53727.878906  ...  49200.703125  6.138568e+10
2636  2021-12-05  49408.492188  ...  48617.156250  3.900810e+10

[2637 rows x 7 columns]


3. Split origianl data into training and testing data




In [20]:
data_training = data[data['Date']<'2021-01-01']
print (data_training)

            Date          Open  ...     Adj Close        Volume
0     2014-09-17    465.864014  ...    457.334015  2.105680e+07
1     2014-09-18    456.859985  ...    424.440002  3.448320e+07
2     2014-09-19    424.102997  ...    394.795990  3.791970e+07
3     2014-09-20    394.673004  ...    408.903992  3.686360e+07
4     2014-09-21    408.084991  ...    398.821014  2.658010e+07
...          ...           ...  ...           ...           ...
2293  2020-12-27  26439.373047  ...  26272.294922  6.647990e+10
2294  2020-12-28  26280.822266  ...  27084.808594  4.905674e+10
2295  2020-12-29  27081.810547  ...  27362.437500  4.526595e+10
2296  2020-12-30  27360.089844  ...  28840.953125  5.128744e+10
2297  2020-12-31  28841.574219  ...  29001.720703  4.675496e+10

[2298 rows x 7 columns]


In [18]:
data_test=  data[data['Date']>='2021-01-01'].copy()
print (data_test)

            Date          Open  ...     Adj Close        Volume
2298  2021-01-01  28994.009766  ...  29374.152344  4.073030e+10
2299  2021-01-02  29376.455078  ...  32127.267578  6.786542e+10
2300  2021-01-03  32129.408203  ...  32782.023438  7.866524e+10
2301  2021-01-04  32810.949219  ...  31971.914063  8.116348e+10
2302  2021-01-05  31977.041016  ...  33992.429688  6.754732e+10
...          ...           ...  ...           ...           ...
2632  2021-12-01  56907.964844  ...  57229.828125  3.685820e+10
2633  2021-12-02  57217.371094  ...  56477.816406  3.237997e+10
2634  2021-12-03  56509.164063  ...  53598.246094  3.978913e+10
2635  2021-12-04  53727.878906  ...  49200.703125  6.138568e+10
2636  2021-12-05  49408.492188  ...  48617.156250  3.900810e+10

[339 rows x 7 columns]


4) Drop irrelavant data for the neural network

In [38]:
training_data= data_training.drop(['Date', 'Adj Close'], axis = 1)
training_data.head()

,Open,High,Low,Close,Volume
0,465.864014,468.174011,452.421997,457.334015,21056800.0
1,456.859985,456.859985,413.104004,424.440002,34483200.0
2,424.102997,427.834991,384.532013,394.795990,37919700.0
3,394.673004,423.295990,389.882996,408.903992,36863600.0
4,408.084991,412.425995,393.181000,398.821014,26580100.0


In [44]:
scaler = MinMaxScaler(feature_range=(0,1))
training_data = scaler.fit_transform(training_data)
print (training_data)

[[1.00809442e-02 8.83276681e-03 1.00216614e-02 9.68757707e-03
  2.04208427e-04]
 [9.76682835e-03 8.44307339e-03 8.61897442e-03 8.54635971e-03
  3.85277136e-04]
 [8.62406341e-03 7.44335417e-03 7.59965585e-03 7.51789714e-03
  4.31621846e-04]
 ...
 [9.38608635e-01 9.35447703e-01 9.20989823e-01 9.43127084e-01
  6.10377732e-01]
 [9.48316726e-01 9.89421170e-01 9.69964757e-01 9.94422366e-01
  6.91583750e-01]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  6.30458660e-01]]


5) Create X and Y training lists and add data into the list

In [63]:
X_train= []
y_train= []

In [62]:
print(training_data.shape[0])

2298


In [ ]:
for i in range(60, training_data.shape[0]):
  X_train.append(training_data[i-60:i])# Append the all the values for every 59 days
  y_train.append(training_data[i,0]) # Append all the opening values for every 60th day



6) Convert Lists into Arrays

In [69]:
X_train, y_train= np.array(X_train), np.array(y_train)

In [70]:
X_train.shape, y_train.shape


((4507, 60, 5), (4507, 5))